In [3]:
# 1. Read the ref-dist-mat into numpy for array
# 2. Replace array names with annotations
# 3. Double for loop to calculate F1 and F2
# 4. Use F measuremnts to calculated M and FST measures

In [4]:
import numpy as np


In [5]:
dist_mat = np.genfromtxt("./data/Clupea_11jan24/11jan_jc_ref-dist-mat.txt.txt", dtype="str")
annots = np.genfromtxt("./data/Clupea_11jan24/11jan_ClupeaAtmore_Fstannot.tsv", dtype="str")
print(annots)
# dist_mat = numpy.genfromtxt("/home/echarvel/Desktop/calab_data/code_bases/Skmer-2/fst_test_data/test_matrix.txt", dtype="str")
# annots = numpy.genfromtxt("/home/echarvel/Desktop/calab_data/code_bases/Skmer-2/fst_test_data/test_annots.txt", dtype="str")

[['genome' 'family']
 ['unclassified-kra_ERR9709092_' 'Unknown']
 ['unclassified-kra_ERR9709093_' 'GieczPoland9th13thcentury']
 ['unclassified-kra_ERR9709096_' 'GieczPoland9th13thcentury']
 ['unclassified-kra_ERR9709100_' 'GieczPoland9th13thcentury']
 ['unclassified-kra_ERR9709101_' 'GieczPoland9th13thcentury']
 ['unclassified-kra_ERR9709105_' 'GieczPoland9th13thcentury']
 ['unclassified-kra_ERR9709107_' 'GieczPoland9th13thcentury']
 ['unclassified-kra_ERR9709109_' 'GieczPoland9th13thcentury']
 ['unclassified-kra_ERR9709114_' 'MalaNieszawkaPoland14th-15thcentury']
 ['unclassified-kra_ERR9709115_' 'MalaNieszawkaPoland14th-15thcentury']
 ['unclassified-kra_ERR9709117_' 'MalaNieszawkaPoland14th-15thcentury']
 ['unclassified-kra_ERR9709118_' 'MalaNieszawkaPoland14th-15thcentury']
 ['unclassified-kra_ERR9709119_' 'MalaNieszawkaPoland14th-15thcentury']
 ['unclassified-kra_ERR9709121_' 'MalaNieszawkaPoland14th-15thcentury']
 ['unclassified-kra_ERR9709122_' 'MalaNieszawkaPoland14th-15thcentury

In [6]:
for line in annots:
    x = np.where(dist_mat == line[0])
    for index in x: 
        try:
            dist_mat[index[0]][index[1]] = line[1]
        except: pass

In [7]:
dist_dict = dict()

for num1 in range(1, len(dist_mat[0])):
    for num2 in range(1, len(dist_mat[0])):
        if num1 != num2: 
            name1=dist_mat[0][num1]
            name2=dist_mat[0][num2]
            if name1 not in dist_dict:
                dist_dict[name1] = dict()

            if name2 not in dist_dict[name1]:
                dist_dict[name1][name2] = [float(dist_mat[num1, num2])]
            else: 
                dist_dict[name1][name2].append(float(dist_mat[num1, num2]))
        if num1 == num2:
            name1=name2=dist_mat[0][num1]
            
            if name1 not in dist_dict:
                dist_dict[name1] = dict()

            if name2 not in dist_dict[name1]:
                dist_dict[name1][name2] = []

key_list = list(dist_dict.keys())

In [8]:
def make_empty_matrix(dist_dict, key_list):
    matrix = [[0 for x in range(0, len(dist_dict)+1)] for x in range(0, len(dist_dict)+1)]
    matrix[0][0]="sample"
    for x in range(0, len(dist_dict)):
        matrix[0][x+1] = matrix[x+1][0] = (key_list[x])
    return matrix

def calculate_WC_FST(F1, F2, M):
    numerator_wc_fst = F1 + F2
    denominator_wc_fst = F1 + F2 + 2 * (1 / (M + 1)) * (M * (1 - F1) + (1 - F2))
    WC_FST = numerator_wc_fst / denominator_wc_fst
    return WC_FST

def calculate_Nei_FST(F1, F2):
    numerator_nei_fst = F1 + F2
    denominator_nei_fst = 2 - ((F1 + F2) / 2)
    Nei_FST = numerator_nei_fst / denominator_nei_fst
    return Nei_FST

def calculate_Hudson_FST(F1, F2):
    Hudson_FST = (F1 + F2) / 2
    return Hudson_FST

In [9]:
F_mtrx = make_empty_matrix(dist_dict, key_list)
wh_fst_mtrx = make_empty_matrix(dist_dict, key_list)
wc_fst_mtrx = make_empty_matrix(dist_dict, key_list)
nei_fst_mtrx = make_empty_matrix(dist_dict, key_list)
hud_fst_mtrx = make_empty_matrix(dist_dict, key_list)
clade_dist_mtrx = make_empty_matrix(dist_dict, key_list)
thetas_list = [[key,0.0] for key in key_list]

In [10]:
for key1, index1 in zip(key_list, range(len(key_list))):
     for key2, index2 in zip(key_list, range(len(key_list))):
        if key1 == key2:
            break
        else:
            dist_array_k1_k1 = dist_dict[key1][key1]
            dist_array_k2_k2 = dist_dict[key2][key2]
            dist_array_k1_k2 = dist_dict[key1][key2]

            across_k1_k2 = sum(dist_array_k1_k2)/len(dist_array_k1_k2)

            max_k1_k1 = max(dist_array_k1_k1) if (len(dist_array_k1_k1) != 0) else 0
            max_k2_k2 = max(dist_array_k1_k1) if (len(dist_array_k1_k1) != 0) else 0
            max_k1_k2 = max(dist_array_k1_k2) 

            clade_dist_mtrx[index1+1][index2+1] = clade_dist_mtrx[index2+1][index1+1] = max_k1_k2 - (max_k1_k1 + max_k2_k2)/2
 
            if ((len(dist_array_k1_k1) == 0) or len(dist_array_k2_k2) == 0):
                # Checks if any of the populations only have one members.
                # Calculates F1 or F2 if possible. If not, assigns "NA". 
                # Assigns "NA" to all other Fst calculations.
                if (len(dist_array_k1_k1) != 0):
                    within_k1 = sum(dist_array_k1_k1)/len(dist_array_k1_k1)
                    F1 = within_k1/across_k1_k2
                    thetas_list[index1] = round(within_k1, 5)
                    F_mtrx[index1+1][index2+1] = round(F1, 5)
                    F_mtrx[index2+1][index1+1] = "NA"
                elif (len(dist_array_k2_k2) != 0):
                    within_k2 = sum(dist_array_k2_k2)/len(dist_array_k2_k2)
                    F2 = within_k2/across_k1_k2
                    thetas_list[index1] = "NA"
                    F_mtrx[index1+1][index2+1] = "NA"
                    F_mtrx[index2+1][index1+1] = round(F2, 5)
                    
                else:
                    F_mtrx[index1+1][index2+1] = F_mtrx[index2+1][index1+1] ="NA"

                wh_fst_mtrx[index1+1][index2+1] = wh_fst_mtrx[index2+1][index1+1] = "NA"
                wc_fst_mtrx[index1+1][index2+1] = wc_fst_mtrx[index2+1][index1+1] = "NA"
                nei_fst_mtrx[index1+1][index2+1] = nei_fst_mtrx[index2+1][index1+1] = "NA"
                hud_fst_mtrx[index1+1][index2+1] = hud_fst_mtrx[index2+1][index1+1] = "NA"
            else:
                within_k1 = sum(dist_array_k1_k1)/len(dist_array_k1_k1)
                within_k2 = sum(dist_array_k2_k2)/len(dist_array_k2_k2)

                F1 = within_k1/across_k1_k2
                F2 = within_k2/across_k1_k2

                if len(dist_array_k1_k1) <= len(dist_array_k2_k2):
                    M=len(dist_array_k1_k1)/len(dist_array_k2_k2)
                else:
                    M=len(dist_array_k2_k2)/len(dist_array_k1_k1)

                thetas_list[index1] = round(within_k1, 5)
                # NOTE: F_matrix contains F1 in the lower half and F2 in the upper half. 
                F_mtrx[index1+1][index2+1] =  round(F1, 5)
                F_mtrx[index2+1][index1+1] =  round(F2, 5)
                wh_fst_mtrx[index1+1][index2+1] = wh_fst_mtrx[index2+1][index1+1] = round((F1 + F2)/2, 5)
                wc_fst_mtrx[index1+1][index2+1] = wc_fst_mtrx[index2+1][index1+1] = round(calculate_WC_FST(F1, F2, M), 5)
                nei_fst_mtrx[index1+1][index2+1] = nei_fst_mtrx[index2+1][index1+1] = round(calculate_Nei_FST(F1, F2), 5)
                hud_fst_mtrx[index1+1][index2+1] = hud_fst_mtrx[index2+1][index1+1] =  round(calculate_Hudson_FST(F1, F2), 5)
print(wh_fst_mtrx)

[['sample', 'Unknown', 'GieczPoland9th13thcentury', 'MalaNieszawkaPoland14th-15thc', 'TrusoJanówPomorskiPoland80085', 'KolowbrzwegBudzistowoPoland75', 'SelsoVestbyDenmark10th11thcen', 'SelsoVestbyDenmark12901380CE', 'KarmoyNorway2002', 'MoreNorway2002', 'RisorNorway2003', 'MaseskarSweden2003', 'KalixFinland2002', 'KalmarsundSweden2010', 'IdefjordInnerNorway2010', 'IdefjordOuterNorway2010'], ['Unknown', 0, 0.60695, 0.75046, 0.61354, 'NA', 0.75272, 0.81353, 0.9281, 'NA', 0.96023, 0.96163, 0.95143, 0.95053, 0.95443, 1.02053], ['GieczPoland9th13thcentury', 0.60695, 0, 0.75055, 0.62321, 'NA', 0.80102, 0.79406, 0.40944, 'NA', 0.45538, 0.40887, 0.45759, 0.45795, 0.42235, 0.50544], ['MalaNieszawkaPoland14th-15thc', 0.75046, 0.75055, 0, 0.76222, 'NA', 0.89704, 0.95419, 0.71174, 'NA', 0.72327, 0.7084, 0.72656, 0.72811, 0.71478, 0.7153], ['TrusoJanówPomorskiPoland80085', 0.61354, 0.62321, 0.76222, 0, 'NA', 0.82458, 0.79335, 0.55337, 'NA', 0.56764, 0.55382, 0.5683, 0.5692, 0.55869, 0.58161], ['Kol

In [11]:
np.savetxt("./testClupeaAtmore/11jan_ClupeaAtmore_theta_list_genu_nas.txt", np.matrix(thetas_list), fmt="%s", delimiter="\t")
np.savetxt("./testClupeaAtmore/11jan_ClupeaAtmore_F_mat_genu_nas.txt", np.matrix(F_mtrx), fmt="%s", delimiter="\t")
np.savetxt("./testClupeaAtmore/11jan_ClupeaAtmore_wh_fst_mat_genu_nas.txt", np.matrix(wh_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("./testClupeaAtmore/11jan_ClupeaAtmore_wc_fst_mat_genu_nas.txt", np.matrix(wc_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("./testClupeaAtmore/11jan_ClupeaAtmore_nei_fst_mat_genu_nas.txt", np.matrix(nei_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("./testClupeaAtmore/11jan_ClupeaAtmore_hud_fst_mat_genu_nas.txt", np.matrix(hud_fst_mtrx), fmt="%s", delimiter="\t")
np.savetxt("./testClupeaAtmore/11jan_ClupeaAtmore_clade_dist_mat_genu_nas.txt", np.matrix(clade_dist_mtrx), fmt="%s", delimiter="\t")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (15,) + inhomogeneous part.